In [ ]:
%%writefile tools.py

import os
from typing import Optional

def search_process(id: str) -> str:
    """
    Procura uma pasta de processo SEI no sistema de arquivos.

    Use esta função para localizar documentos de processos administrativos pelo seu número de referência.
    A função lida com formatos de ID tradicionais (XXX/YYYY) e compactos (XXXYYYY).

    Args:
        id (str): Número do processo em qualquer formato:
            - Formato separado: "XXX/YYYY"
            - Formato compacto: "XXXYYYY"
            O número será automaticamente preenchido com zeros à esquerda se necessário.

    Returns:
        str: Um dos seguintes:
            - Nome da pasta se o processo existir
            - None se o processo não for encontrado
            - "Process not found" se o processo não for encontrado (formato separado ou erros)
    """
    root_path = os.path.abspath("")
    processes_path = os.path.join(root_path, "processos")
    try:
        if len(id) < 9:
            if id.find("/") == -1:
                id = id.zfill(9)
            else:
                id = id.split("/")
                id[0] = id[0].zfill(5)
                id[1] = id[1]
                id = "/".join(id)
        if id.find("/") == -1:
            folder = f"SEI_{id[:-4]}_{id[-4:]}"
            # print(f"Searching for {folder}")
            if os.path.exists(os.path.join(processes_path, folder)):
                # print(f"Process {id} found!")
                return folder
            else:
                # print(f"Process {id} not found!")
                return "Process not found"
        else:
            folder = f"SEI_{id.split('/')[0]}_{id.split('/')[1]}"
            # print(f"Searching for {folder}")
            if os.path.exists(os.path.join(processes_path, folder)):
                # print(f"Process {id} found!")
                return folder
            else:
                # print(f"Process {id} not found!")
                return "Process not found"
    except Exception as e:
        print(f"Error: {e}")
        return "Process not found"

def get_document_list_from_process(
    parameters: str
) -> list[str]:
    """
    Recupera documentos PDF de uma pasta de processo SEI com suporte a paginação.

    Use esta função para obter uma lista de documentos PDF dentro de uma pasta de processo.
    Os resultados podem ser paginados usando parâmetros de limite e deslocamento.
    Normalmente usado após localizar uma pasta de processo com search_process().

    Args:
        parameters (str): Uma string contendo o nome da pasta do processo e parâmetros de paginação.
            A string deve ser formatada da seguinte forma:
            "process_folder,limit,offset"
            - process_folder: O nome da pasta do processo
            - limit: O número máximo de documentos a retornar
            - offset: O número de documentos a pular

    Returns:
        Union[dict(str : list[str], str : int), str]: Um dos seguintes:
            - Um dicionário contendo:
                - "documents": Uma lista de nomes de documentos PDF
                - "total_number_of_documents": O número total de documentos na pasta
            - "Invalid parameters" se a string de entrada não estiver formatada corretamente
            - "Process folder not found" se a pasta do processo não existir
    """
    try:
        process_folder, limit, offset = parameters.split(",")
        limit = int(limit)
        offset = int(offset)
    except Exception as e:
        print(f"Error: {e}")
        return "Invalid parameters"
    try:
        tree = os.walk(os.path.join(os.path.abspath(""), "processos", process_folder))
        documents = []
        for root, dirs, files in tree:
            documents.extend([
                file
                for file in files
                if file.endswith(".pdf")
                ])
        documents.sort()
        return {
            "documents" : documents[offset:offset+limit],
            "total_number_of_documents" : len(documents)
        }
    except Exception as e:
        print(f"Error: {e}")
        return "Process folder not found"

def read_doc(file_path: str) -> Optional[str]:
    """
    Lê um documento PDF e extrai seu conteúdo de texto.

    Args:
        file_path (str): O caminho para o arquivo PDF.

    Returns:
        Optional[str]: O texto extraído do PDF, ou None se ocorrer um erro.
    """
    from PyPDF2 import PdfReader
    try:
        reader = PdfReader(file_path)
        contents = []
        for i in range(len(reader.pages)):
            page = reader.pages[i]
            contents.append(page.extract_text())
        return "\n".join(contents)
    except Exception as e:
        return f"Error: {e}"
    
def get_document_by_type(parameters : str) -> str:
    """
    Obtem uma lista de documentos por tipo de um processo SEI.
    
    Args:
        parameters (str): Uma string contendo o ID do processo e tipo de documento.
            A string deve ser formatada da seguinte forma:
            "process_id,document_type"
            - process_id: O ID do processo (ex: "XXX/YYYY" ou "XXXYYYY") 
            - document_type: O tipo de documento a ser procurado
    """
    process_id , document_type = parameters.split(",")
    process_folder = search_process(process_id)
    if process_folder == "Process not found" or process_folder == "Process folder not found":
        return process_folder
    response = get_document_list_from_process(f"{process_folder},1,0")
    total_documents = response["total_number_of_documents"]
    response = get_document_list_from_process(f"{process_folder},{total_documents},0")
    documents = response["documents"]
    documents_found = [
        document
        for document in documents
        if document_type.lower() in document.lower()
    ]
    if len(documents_found) > 0:
        return {
            "documents" : documents_found,
            "number_of_documents" : len(documents_found)
        }
    else:
        return f"Documents of type {document_type} not found in process {process_id}"

In [ ]:
%%writefile MultiAgent.py

import subprocess

from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

from tools import *

class MultiAgents:
    def __init__(self, models):
        self.supervisor_model, self.agent_model = self.initialize_models(models)
        # self.chat_agent = self.initialize_agent(
        #     name="chat_agent",
        #     tools=[],
        #     prompt="You can only chat with the user."
        # )
        self.research_agent = self.initialize_agent(
            name="sei_research_agent",
            tools=[search_process, get_document_list_from_process, get_document_by_type],
            prompt=(
                "Você é especialista em obter informações sobre processos do SEI. "
                "Você é capaz de: "
                "pesquisar processos usando a função search_process, "
                "listar documentos de um processo usando a função get_document_list_from_process, "
                "e obter tipos específicos de documentos de um processo usando a função get_document_by_type. "
            )
        )
        self.graph = self.initialize_graph((
            "Você é um chatbot com um time de especialistas para atender o usuário. "
            "Use sei_research_agent para responder perguntas sobre processos no "
            "Sistema Eletrônico de Informações (SEI) do TRE do Rio Grande do Norte (TRE-RN)."
        ))

    def initialize_models(self, models):
        match models['supervisor']["provider"]:
            case "groq":
                print("Using Groq model for supervisor")
                llm_supervisor = ChatGroq(
                    model=models['supervisor']['model'],
                    temperature=models['supervisor']['temperature']
                )
            case "ollama":
                print("Using Ollama model for supervisor")
                llm_supervisor = ChatOllama(
                    model=models['supervisor']['model'],
                    temperature=models['supervisor']['temperature']
                    )
                model_name = models['supervisor']['model']
                subprocess.run(["ollama", "pull", model_name])
                
        match models['agent']['provider']:
            case "groq":
                print("Using Groq model for agent")
                llm_agent = ChatGroq(
                    model=models['agent']['model'],
                    temperature=models['agent']['temperature']
                )
            case "ollama":
                print("Using Ollama model for agent")
                llm_agent = ChatOllama(
                    model=models['agent']['model'],
                    temperature=models['agent']['temperature']
                    )
                model_name = models['agent']['model']
                subprocess.run(["ollama", "pull", model_name])
        return llm_supervisor, llm_agent
    
    def initialize_agent(self, name, prompt, tools):
        agent =  create_react_agent(
            self.agent_model,
            name=name,
            tools=tools,
            prompt=prompt,
        )
        return agent
    
    def initialize_graph(self, prompt):
        workflow = create_supervisor(
            [self.research_agent],
            model=self.supervisor_model,
            prompt=prompt,
            output_mode="full_history"
        )
        return workflow.compile()
    
    def run(self, messages, recursion_limit=10):
        return self.graph.invoke(
            messages,
            {"recursion_limit": recursion_limit}
        )
    
    def stream(self, messages, recursion_limit=10):
        return self.graph.stream(
            messages,
            {"recursion_limit": recursion_limit},
            stream_mode="values"
        )

In [ ]:
%%writefile Chatbot.py

import streamlit as st
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
import weave
from typing import Optional, Tuple

from MultiAgent import MultiAgents

import logging, os
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_dotenv(
    override=True
)

logging.info("Loaded environment variables")
logging.info(f"GROQ_API_KEY: {os.getenv('GROQ_API_KEY')}")
logging.info(f"WANDB_API_KEY: {os.getenv('WANDB_API_KEY')}")

# Initialize Weave
weave.init("streamlit_sei")

def initialize_session_state():
    """Initialize the session state with a welcome message."""
    if "messages" not in st.session_state:
        st.session_state['messages'] = [
            # {
            #     "role": "assistant",
            #     "content": "Olá! Como posso ajudar você com o Sistema Eletrônico de Informações (SEI)?"
            # }
        ]

def get_message(message_data) -> Optional[Tuple[str, str]]:
    """
    Process the message and return a tuple of (role, content).
    
    Args:
        message_data: The message data from the stream
        
    Returns:
        Tuple containing (role, content) or None if message should be skipped
    """
    message = message_data["messages"][-1]
    
    if isinstance(message, (HumanMessage, AIMessage, ToolMessage)):
        role = "user" if isinstance(message, HumanMessage) else "assistant"
        return role, message.content
    return None

def setup_agents():
    """Configure and return the MultiAgents setup."""
    models = {
        "supervisor": {
            "provider": "groq",
            "model": "llama-3.3-70b-versatile",
            "temperature": 0.0
        },
        "agent": {
            "provider": "groq",
            "model": "llama3-8b-8192",
            "temperature": 0.0
        },
    }
    return MultiAgents(models)

def should_display_message(content: str) -> bool:
    """
    Check if the message should be displayed in the chat.
    """
    skip_phrases = [
        "Successfully transferred",
        "transferred to",
        "transferred back"
    ]
    return not any(phrase in content for phrase in skip_phrases)

def main():
    # Page configuration
    st.title("💬 Chatbot SEI TRE-RN")
    st.caption("Um chatbot para responder perguntas sobre processos no Sistema Eletrônico de Informações (SEI) do Tribunal Regional Eleitoral do Rio Grande do Norte (TRE-RN).")
    
    # Initialize session state
    initialize_session_state()
    
    # Setup agents
    agents = setup_agents()
    
    # Display chat history
    for msg in st.session_state['messages']:
        st.chat_message(msg["role"]).write(msg["content"])
    
    # Handle user input
    if prompt := st.chat_input():
        # Add user message to chat
        st.session_state.messages.append({"role": "user", "content": prompt})
        st.chat_message("user").write(prompt)
        
        try:
            # Stream the response
            with st.spinner("Processando sua pergunta..."):
                stream = agents.stream({"messages": st.session_state['messages']})
                
                # Create a placeholder for the assistant's message
                message_placeholder = st.chat_message("assistant")
                full_response = ""
                assistant_content = ""
                
                # Add logging for debugging
                logging.info(f"Starting to process stream for prompt: {prompt[:30]}...")
                
                for stream_data in stream:
                    result = get_message(stream_data)
                    if result is None:
                        continue
                    
                    role, content = result
                    logging.debug(f"Received message - Role: {role}, Content length: {len(content)}")
                    
                    if role == "assistant" and should_display_message(content):
                        # For assistant messages, always use the latest complete chunk
                        # This prevents partial messages from being displayed
                        assistant_content = content
                        # Update the displayed message with complete content
                        message_placeholder.markdown(assistant_content)
                
                # Only append the final response to session state
                if assistant_content:
                    logging.info(f"Final response length: {len(assistant_content)}")
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": assistant_content
                    })
                else:
                    logging.warning("No assistant content was generated!")
                    
        except Exception as e:
            logging.error(f"Error during streaming: {str(e)}")
            st.error(f"Ocorreu um erro: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
!pip install -q streamlit
!npm install localtunnel

In [ ]:
!streamlit run Chatbot.py &>/content/logs.txt &

In [ ]:
!curl ipv4.icanhazip.com

In [ ]:
!npx localtunnel --port 8501